## Leer el conjunto de datos y tratar valores faltantes.

In [1]:
# Importar Pandas
import pandas as pd

# Importar librerías para visualizaciones
import seaborn as sns
import matplotlib.pyplot as plt 

# Mejorar calidad de los gráficos
sns.set_style("whitegrid")
from matplotlib_inline import backend_inline 
backend_inline.set_matplotlib_formats("svg")

In [2]:
# Guardar conjunto de datos en la variable df.
df = pd.read_csv("../DATA/house_tiny.csv")
df

,num_rooms,ciudad,price
0,1.0,atenas,'86.16929456124842'
1,1.0,atenas,'123.46886436842463'
2,2.0,atenas,'183.9242357698576'
3,2.0,atenas,'208.1115355978664'
4,4.0,atenas,'401.6046221689021'
...,...,...,...
95,2.0,madrid,'302.32449241557737'
96,1.0,madrid,'145.54056269313887'
97,2.0,madrid,'310.74500783078577'
98,NaN,madrid,'568.9694705980224'


In [3]:
# Información muy bien resumida de nuestro conjunto de datos.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   num_rooms  98 non-null     float64
 1   ciudad     100 non-null    object 
 2   price      100 non-null    object 
dtypes: float64(1), object(2)
memory usage: 2.5+ KB


In [11]:
# Tenemos cadenas de texto en la columna price
df["price"].sample(5)

99    '189.76107399919013'
10     '93.39962360970732'
1     '123.46886436842463'
78    '150.75657537418633'
51     '400.1912337615752'
Name: price, dtype: object

In [12]:
# Solucionar problema de tipo de dato en columna price
df["price"] = df["price"].apply(lambda precio: float(precio.replace("'", "")))
df

,num_rooms,ciudad,price
0,1.0,atenas,86.169295
1,1.0,atenas,123.468864
2,2.0,atenas,183.924236
3,2.0,atenas,208.111536
4,4.0,atenas,401.604622
...,...,...,...
95,2.0,madrid,302.324492
96,1.0,madrid,145.540563
97,2.0,madrid,310.745008
98,NaN,madrid,568.969471


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   num_rooms  98 non-null     float64
 1   ciudad     100 non-null    object 
 2   price      95 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [14]:
# Estadística descriptiva de columnas numéricas.
df.describe()

,num_rooms,price
count,98.000000,95.000000
mean,1.785714,278.254423
std,0.997419,181.555708
min,1.000000,35.805225
25%,1.000000,155.956713
50%,1.000000,212.037848
75%,2.000000,383.966020
max,4.000000,849.506297


In [15]:
# Máscara de Trues y Falses indicando donde hay valores nulos
df.isnull()

,num_rooms,ciudad,price
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
95,False,False,False
96,False,False,False
97,False,False,False
98,True,False,False


In [16]:
# Número de valores nulos por columnas
df.isnull().sum(axis=0)

num_rooms    2
ciudad       0
price        5
dtype: int64

In [18]:
# ¿Qué filas tienen valores nulos?
df.isnull().sum(axis=1)>0

0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98     True
99    False
Length: 100, dtype: bool

In [19]:
null_rows = df[df.isnull().sum(axis=1)>0].copy()
null_rows

,num_rooms,ciudad,price
15,2.0,atenas,NaN
30,NaN,atenas,229.845542
37,2.0,berlin,NaN
47,1.0,berlin,NaN
87,1.0,madrid,NaN
94,4.0,madrid,NaN
98,NaN,madrid,568.969471


<span style="font-size:20px">

- Proceso para resolver valores NaN de precio: 
    - Obtener precio mediano de la ciudad que corresponda.
    - Rellenar NaN value con dicho precio mediano.

In [20]:
def get_median_price_city(ciudad):
    return df[df["ciudad"] == ciudad]["price"].median()

In [22]:
get_median_price_city("madrid")

238.47058131536693

In [24]:
import numpy as np 
pd.isna(np.nan)

True

In [28]:
for row in null_rows.iterrows():
    break 
row[1]["ciudad"]

'atenas'

In [29]:
# Función para iterar sobre las filas con NaN
def solve_nan_price(row):
    # Si el valor de Price es un NaN value
    if pd.isna(row["price"]):
        # Obtener ciudad de la fila
        ciudad = row["ciudad"]
        # Calcular mediana de la ciudad
        median_price_ciudad = get_median_price_city(ciudad)
        # Rellenar NaN con mediana
        row["price"] = median_price_ciudad 
    # Devolver el row
    return row 

null_rows = null_rows.apply(solve_nan_price, axis=1)
null_rows

,num_rooms,ciudad,price
15,2.0,atenas,123.468864
30,NaN,atenas,229.845542
37,2.0,berlin,230.448343
47,1.0,berlin,230.448343
87,1.0,madrid,238.470581
94,4.0,madrid,238.470581
98,NaN,madrid,568.969471


In [31]:
df.iloc[null_rows.index] = null_rows
df.iloc[null_rows.index]

,num_rooms,ciudad,price
15,2.0,atenas,123.468864
30,NaN,atenas,229.845542
37,2.0,berlin,230.448343
47,1.0,berlin,230.448343
87,1.0,madrid,238.470581
94,4.0,madrid,238.470581
98,NaN,madrid,568.969471


In [32]:
df.isnull().sum()

num_rooms    2
ciudad       0
price        0
dtype: int64

In [36]:
df = df.dropna().reset_index(drop=True)
df

,num_rooms,ciudad,price
0,1.0,atenas,86.169295
1,1.0,atenas,123.468864
2,2.0,atenas,183.924236
3,2.0,atenas,208.111536
4,4.0,atenas,401.604622
...,...,...,...
93,4.0,madrid,238.470581
94,2.0,madrid,302.324492
95,1.0,madrid,145.540563
96,2.0,madrid,310.745008


In [37]:
df.isnull().sum()

num_rooms    0
ciudad       0
price        0
dtype: int64

## Visualizaciones y valores atípicos.

In [ ]:
# Correlación entre variables numéricas
df.corr(numeric_only=True)

In [ ]:
correlation = df.corr(numeric_only=True)["num_rooms"]["price"]
sns.scatterplot(x=df["num_rooms"], y=df["price"])
plt.title(f"Correlación: {correlation:.2f}")

In [ ]:
# En este ejemplo quizás no es muy útil...
sns.scatterplot(x=df["num_rooms"], y=df["price"], hue=df["ciudad"])

In [ ]:
df["ciudad"].value_counts()

In [ ]:
sns.barplot(x=df["ciudad"].value_counts().index, y=df["ciudad"].value_counts())

In [ ]:
info_ciudades = {"Ciudad": [],
                 "Precio_medio": [],
                 "Num_rooms_medio": []}
for ciudad in df["ciudad"].unique():
    mean_price = df[df["ciudad"] == ciudad]["price"].mean()
    mean_num_rooms = df[df["ciudad"] == ciudad]["num_rooms"].mean()
    info_ciudades["Ciudad"].append(ciudad)
    info_ciudades["Precio_medio"].append(mean_price)
    info_ciudades["Num_rooms_medio"].append(mean_num_rooms)

info_ciudades = pd.DataFrame(info_ciudades)
info_ciudades = info_ciudades.sort_values(by="Precio_medio", ascending=False)
info_ciudades

In [ ]:
info_ciudades = df.groupby("ciudad").mean().reset_index()
info_ciudades = info_ciudades.rename(columns={"num_rooms": "mean_num_rooms",
                              "price": "mean_price"})
info_ciudades = info_ciudades.sort_values(by="mean_price", ascending=False)
info_ciudades

In [ ]:
sns.barplot(x=info_ciudades["ciudad"], y=info_ciudades["mean_price"])

In [ ]:
sns.boxplot(df["price"])

<span style="font-size:20px">

- **Valores atípicos**: instancia que se desvía significativamente del resto de datos del conjunto
- ¿Qué signifca significativamente?
- Pues, el método de Tukey realiza los siguientes pasos para detectar un *valor atípico*.
    - Calcular cuartiles 1 y 3. 
        - Q1: valor que verifica que hay un 25% de los datos que son menores.
        - Q3: valor que verifica que hay un 75% de los datos que son menores.
    - Calcular rango intercuartílico $$IQR=Q3-Q1$$
    - Definir límites para los valores atípicos
        - **Límite superior**: $Q3 + 1.5\cdot IQR$ 
        - **Límite inferior**: $Q1 - 1.5\cdot IQR$ 


In [ ]:
def get_outliers(numeric_column, constant=1.5):
    Q1 = numeric_column.describe()["25%"]
    Q3 = numeric_column.describe()["75%"]
    iqr = Q3 - Q1 
    superior = Q3 + constant*iqr 
    inferior = Q1 - constant*iqr 
    where_outliers = (numeric_column<inferior) | (superior<numeric_column)
    return numeric_column[where_outliers]

In [ ]:
get_outliers(df["price"])

In [ ]:
# Podemos eliminar outliers de forma sencilla
df.drop(get_outliers(df["price"]).index)